# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import random
import math
import json
from collections import defaultdict
from tqdm.notebook import tqdm
from IPython.display import clear_output
import zipfile
import os
from google.colab import files
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn.utils.prune as prune

# Rendering
from skimage.measure import marching_cubes
from IPython.display import Image as IPImage

In [ ]:
try:
    !pip install trimesh
    import trimesh
    TRIMESH_AVAILABLE = True
except ImportError:
    print("trimesh not available, using fallback for mesh operations.")
    TRIMESH_AVAILABLE = False

try:
    import scipy
    from scipy.spatial import KDTree
    SCIPY_AVAILABLE = True
except ImportError:
    print("Warning: scipy not available, using fallback distance computation.")
    SCIPY_AVAILABLE = False
!pip install scikit-image
SKLEARN_AVAILABLE = True  

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")

# Globals

In [ ]:
DATASET_NAME = "synthetic_rooms"  # Using real synthetic_rooms dataset, otherwise "shapenet"

In [ ]:
# Dataset configurations
DATASET_CONFIGS = {
    "shapenet": {
        "path": "/kaggle/input/subdataset-random-pick-shapenet-5-gb/ShapeNet",
        "ground_truth_path": None,  # Ground truth in same directory
        "point_cloud_size": 2048,
        "sample_points": 4096,
        "categories": None,  # none loads all categories, otherwise specify
        "file_extension": ".npz",
        "normalize": True
    },
    "synthetic_rooms": {
        "path": "/kaggle/input/synthetic-rooms-rooms-04050607/synthetic rooms",
        "ground_truth_path": "/kaggle/input/gt-synthetic-rooms-04050607/gt_rooms_mesh",
        "point_cloud_size": 2048,
        "sample_points": 4096,
        "file_extension": ".npz",
        "normalize": True
    }
}

In [ ]:
# Get current dataset config
CURRENT_DATASET = DATASET_CONFIGS[DATASET_NAME]

# Training parameters
BATCH_SIZE = 64
INPUT_DIM = 3
OUTPUT_DIM = 1
POINT_CLOUD_SIZE = CURRENT_DATASET["point_cloud_size"]
SAMPLE_POINTS = CURRENT_DATASET["sample_points"]

# Model parameters
ENCODER_C_DIM = 32
PLANE_RESOLUTION = 32

LEARNING_RATE = 5e-4
NUM_EPOCHS = 100 # Increased from 25 to 100
DROPOUT_RATE = 0.2  # Doubled for decoder

# Weighted sampler
POSITIVE_WEIGHT_MULTIPLIER = 10.0     # 1.0 to disable completely weighted sampling

# Paths
DATA_PATH = CURRENT_DATASET["path"]
MODEL_SAVE_PATH = f"./models/{DATASET_NAME}"
RESULTS_PATH = f"./results/{DATASET_NAME}"

# Extra : model pruning and fine-tuning
PRUNING_RATIO = 0.07
FINETUNE_EPOCHS = 30
FINETUNE_LR = 1e-6
FINE_TUNING_PATIENCE = 10

In [ ]:
# Create directories if they don't exist
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# Utils

In [ ]:
def set_seed(seed=42):
    """Set random seed for reproducibility"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

def count_parameters(model):
    """Count the number of trainable parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def normalize_point_cloud(pc):
    """Normalize point cloud to unit sphere"""
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    max_dist = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / max_dist
    return pc

In [ ]:
def save_checkpoint(model, optimizer, epoch, loss, path):
    """Save model checkpoint"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'loss': loss,
    }

    if optimizer is not None:
        checkpoint['optimizer_state_dict'] = optimizer.state_dict()

    torch.save(checkpoint, path)
    print(f"Checkpoint saved at {path}")

def load_checkpoint(model, optimizer, path):
    """Load model checkpoint"""
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    if optimizer is not None and 'optimizer_state_dict' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
    return checkpoint['epoch'], checkpoint['loss']

In [ ]:
def validate_dataset_path():
    """Validate that the dataset path exists"""
    if not os.path.exists(DATA_PATH):
        raise FileNotFoundError(f"Dataset path {DATA_PATH} does not exist! Ensure the real dataset is available.")
    return True

In [ ]:
def get_dataset_info():
      """Display current dataset configuration information"""
      print(f"Current dataset: {DATASET_NAME}")
      print(f"Dataset path: {DATA_PATH}")
      print(f"Point cloud size: {POINT_CLOUD_SIZE}")
      print(f"Sample points: {SAMPLE_POINTS}")

## Setup

In [ ]:
print("=" * 70)
print("TransONet Convolutional Occupancy Networks for Virtual Scene Generation")
print("=" * 70)

set_seed(42)
print("Random seed set")
get_dataset_info()

# Data

In [ ]:
def create_weighted_sampler(dataset, positive_weight_multiplier=10.0):
    from torch.utils.data import WeightedRandomSampler
    
    sample_weights = []
    occupied_samples = 0
    
    for i in tqdm(range(len(dataset)), desc="Computing weights"):
        sample = dataset[i]
        occupancy = sample['occupancy']
        
        if (occupancy > 0.5).any().item():
            positive_ratio = (occupancy > 0.5).float().mean().item()
            weight = positive_weight_multiplier * (1.0 + positive_ratio)
            occupied_samples += 1
        else:
            weight = 1.0
        sample_weights.append(weight)
    
    return WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

def create_data_loaders(batch_size=BATCH_SIZE, use_weighted_sampler=True, positive_weight_multiplier=10.0):
    train_dataset = BaseDataset(DATA_PATH, 'train')
    val_dataset = BaseDataset(DATA_PATH, 'val')

    if use_weighted_sampler:
        train_sampler = create_weighted_sampler(train_dataset, positive_weight_multiplier)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=2)
    else:
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    return train_loader, val_loader

In [ ]:
class BaseDataset(Dataset):
    """Base dataset class for real datasets only"""

    def __init__(self, data_path, split='train', num_sample_points=SAMPLE_POINTS, dataset_type=DATASET_NAME):
        self.data_path = data_path
        self.split = split
        self.num_sample_points = num_sample_points
        self.dataset_type = dataset_type
        self.config = CURRENT_DATASET
        self.has_real_data = os.path.exists(self.data_path)
        
        # Ground truth path for mesh evaluation
        self.ground_truth_path = self.config.get("ground_truth_path", None)
        self.has_ground_truth = self.ground_truth_path and os.path.exists(self.ground_truth_path)

        # Validate dataset exists - fail if not found
        validate_dataset_path()

        # Load data file paths
        self.data_files = self._load_data_files()

        if not self.data_files:
            raise ValueError(f"No data files found for {dataset_type} dataset in {data_path}")

    def _load_data_files(self):
        """Load list of data files based on dataset type"""
        if self.dataset_type == "shapenet":
            return self._load_shapenet_files()
        elif self.dataset_type == "synthetic_rooms":
            return self._load_synthetic_rooms_files()
        else:
            raise ValueError(f"Unknown dataset type: {self.dataset_type}")

    def _load_shapenet_files(self):
        """Load ShapeNet file list"""
        files = []

        if self.has_real_data:
            # Get categories if None, discover all avaiable
            categories = self.config["categories"]
            if categories is None:
                categories = [d for d in os.listdir(self.data_path)
                              if os.path.isdir(os.path.join(self.data_path, d))]

        # Load from actual ShapeNet structure
        for category in categories:
            category_path = os.path.join(self.data_path, category)
            if os.path.exists(category_path):
                split_file = os.path.join(category_path, f"{self.split}.lst")
                if os.path.exists(split_file):
                    with open(split_file, 'r') as f:
                        category_files = [f"{category}/{line.strip()}" for line in f.readlines()]
                        files.extend(category_files)
                else:
                    # If no split file, use all objects in category
                    for obj_id in os.listdir(category_path):
                        obj_path = os.path.join(category_path, obj_id)
                        if os.path.isdir(obj_path):
                            files.append(f"{category}/{obj_id}")

        # Apply train/val split if no predefined split files exist
        if files and not any(os.path.exists(os.path.join(self.data_path, cat, f"{self.split}.lst"))
                           for cat in categories):
            # Random split: 80% train, 20% val
            np.random.seed(42)
            np.random.shuffle(files)
            split_idx = int(0.8 * len(files))

            if self.split == 'train':
                files = files[:split_idx]
            else:
                files = files[split_idx:]

        return files

    def _load_synthetic_rooms_files(self):
        """Load Synthetic Rooms file list with correct directory structure"""
        files = []

        # Look for rooms_XX directories
        for room_class in os.listdir(self.data_path):
            room_path = os.path.join(self.data_path, room_class)
            if os.path.isdir(room_path) and room_class.startswith('rooms_'):
                # Look for scene directories (00000000, 00000001, etc.)
                for scene_id in os.listdir(room_path):
                    scene_path = os.path.join(room_path, scene_id)
                    if os.path.isdir(scene_path):
                        # Check if required subdirectories exist
                        pointcloud_dir = os.path.join(scene_path, 'pointcloud')
                        points_iou_dir = os.path.join(scene_path, 'points_iou')

                        if os.path.exists(pointcloud_dir) and os.path.exists(points_iou_dir):
                            files.append(f"{room_class}/{scene_id}")

        # Apply train/val split
        np.random.seed(42)  # Fixed seed for consistent splits
        np.random.shuffle(files)
        split_idx = int(0.8 * len(files))

        if self.split == 'train':
            files = files[:split_idx]
        else:
            files = files[split_idx:]

        return files

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        """Get a single data sample - real data only"""
        file_id = self.data_files[idx]

        try:
            return self._load_real_data(file_id)
        except Exception as e:
            raise RuntimeError(f"Failed to load real data for {file_id}: {str(e)}. "
                             f"Ensure the dataset is properly formatted.")

    def _load_real_data(self, file_id):
        """Load real dataset file"""
        if self.dataset_type == "shapenet":
            return self._load_shapenet_sample(file_id)
        elif self.dataset_type == "synthetic_rooms":
            return self._load_synthetic_rooms_sample(file_id)

    def _load_shapenet_sample(self, file_id):
        """Load ShapeNet sample"""
        parts = file_id.split('/')
        category = parts[0]
        hash_id = parts[1]

        # Check nested structure first
        nested_path = os.path.join(self.data_path, category, hash_id)
        file_path = None

        if os.path.isdir(nested_path):
            # Look for .npz files inside the hash directory
            for file in os.listdir(nested_path):
                if file.endswith('.npz'):
                    file_path = os.path.join(nested_path, file)
                    break
        if file_path is None:
            # Fallback to direct file
            file_path = os.path.join(self.data_path, category, f"{hash_id}.npz")

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"ShapeNet file not found: {file_path}")

        # Load the file
        data = np.load(file_path)

        # Extract required fields
        if 'points' not in data:
            raise ValueError(f"No 'points' field found in {file_path}")

        point_cloud = data['points'][:self.config["point_cloud_size"]]

        if len(data['points']) == len(data['occupancies']):
            sample_points = data['points'][:self.num_sample_points]
            occupancy_values = data['occupancies'][:self.num_sample_points]
        else:
            # Generate sample points
            raise ValueError(f"Mismatched data shapes in {file_path}")

        if self.config["normalize"]:
            point_cloud = normalize_point_cloud(point_cloud)
            sample_points = normalize_point_cloud(sample_points)

        return {
            'point_cloud': torch.FloatTensor(point_cloud),
            'sample_points': torch.FloatTensor(sample_points),
            'occupancy': torch.FloatTensor(occupancy_values)
        }

    def _load_synthetic_rooms_sample(self, file_id):
        """Load Synthetic Rooms sample with correct file structure"""
        room_class, scene_id = file_id.split('/')
        scene_path = os.path.join(self.data_path, room_class, scene_id)

        # Load pointcloud data
        pointcloud_dir = os.path.join(scene_path, 'pointcloud')
        pointcloud_files = [f for f in os.listdir(pointcloud_dir) if f.endswith('.npz')]
        if not pointcloud_files:
            raise FileNotFoundError(f"No pointcloud files in {pointcloud_dir}")

        # Use first pointcloud file
        local_rng = np.random.RandomState(hash(file_id) % 2**32)
        pointcloud_path = os.path.join(pointcloud_dir, local_rng.choice(pointcloud_files))
        pointcloud_data = np.load(pointcloud_path)

        # Load points_iou data
        points_iou_dir = os.path.join(scene_path, 'points_iou')
        iou_files = [f for f in os.listdir(points_iou_dir) if f.endswith('.npz')]
        if not iou_files:
            raise FileNotFoundError(f"No IoU files in {points_iou_dir}")

        # Use first IoU file
        iou_path = os.path.join(points_iou_dir, local_rng.choice(iou_files))
        iou_data = np.load(iou_path)

        # Extract data with correct field names
        pc_indices = np.random.choice(len(pointcloud_data['points']), self.config["point_cloud_size"], replace=False)
        point_cloud = pointcloud_data['points'][pc_indices]
        indices = np.random.choice(len(iou_data['points']), self.num_sample_points, replace=False)
        sample_points = iou_data['points'][indices]

        # Unpack occupancy values (they're packed as bits)
        packed_occupancies = iou_data['occupancies']
        occupancy_values = np.unpackbits(packed_occupancies).astype(np.float32)
        occupancy_values = occupancy_values[indices]

        if self.config["normalize"]:
            point_cloud = normalize_point_cloud(point_cloud)
            sample_points = normalize_point_cloud(sample_points)

        # Prepare return data
        sample_data = {
            'point_cloud': torch.FloatTensor(point_cloud),
            'sample_points': torch.FloatTensor(sample_points),
            'occupancy': torch.FloatTensor(occupancy_values),
            'file_id': file_id  # Add file_id for ground truth lookup
        }

        # Load ground truth mesh if available
        if self.has_ground_truth:
            gt_file = os.path.join(self.ground_truth_path, room_class, f"{scene_id}.obj")
            if os.path.exists(gt_file):
                sample_data['ground_truth_mesh_path'] = gt_file

        return sample_data

    def get_ground_truth_mesh_path(self, file_id):
        """Get ground truth mesh path for a specific file_id"""
        if not self.has_ground_truth:
            return None
        
        room_class, scene_id = file_id.split('/')
        gt_file = os.path.join(self.ground_truth_path, room_class, f"{scene_id}.obj")
        
        if os.path.exists(gt_file):
            return gt_file
        
        return None

## Data Loading

In [ ]:
print("\n2. DATA LOADING")
print("-" * 40)

train_loader, val_loader = create_data_loaders(
    batch_size=BATCH_SIZE, 
    use_weighted_sampler=True, 
    positive_weight_multiplier=POSITIVE_WEIGHT_MULTIPLIER
)

print(f"✓ Train: {len(train_loader.dataset)} samples (weighted sampling)")
print(f"✓ Val: {len(val_loader.dataset)} samples")

# Network

In [ ]:
class ResNetBlock(nn.Module):
    """ResNet block for PointNet encoder"""
    def __init__(self, c_dim, dropout_rate=0.1):
        super().__init__()
        self.fc1 = nn.Linear(c_dim, c_dim)
        self.fc2 = nn.Linear(c_dim, c_dim)
        self.bn1 = nn.BatchNorm1d(c_dim)
        self.bn2 = nn.BatchNorm1d(c_dim)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        # x: (B, N, C)
        residual = x

        # Apply linear layer
        x = self.fc1(x)

        # BatchNorm1d
        x = x.transpose(1, 2)
        x = F.relu(self.bn1(x))
        x = x.transpose(1, 2)
        x = self.dropout(x)

        # Second layer
        x = self.fc2(x)
        x = x.transpose(1, 2)

        x = self.bn2(x)
        x = x.transpose(1, 2)
        return F.relu(x + residual)

In [ ]:
class ResNetPointNet(nn.Module):
    """ResNet PointNet encoder as required by TransONet"""
    def __init__(self, c_dim=32, input_dim=3, dropout_rate=0.1):
        super().__init__()
        self.c_dim = c_dim
        
        # Initial feature extraction
        self.fc_start = nn.Linear(input_dim, c_dim)
        self.dropout = nn.Dropout(dropout_rate)
        
        # 5 ResNet blocks as specified in the paper
        self.resnet_blocks = nn.ModuleList([
            ResNetBlock(c_dim, dropout_rate) for _ in range(5)
        ])
        
        # Final ResNet block
        self.final_block = ResNetBlock(c_dim, dropout_rate)
        
    def forward(self, x):
        batch_size, num_points, _ = x.shape
        
        # Initial features
        x = self.fc_start(x)
        
        # Pass through ResNet blocks
        for block in self.resnet_blocks:
            x = block(x)
            
        # Final block
        x = self.final_block(x)
        
        return x

In [ ]:
class DynamicPlanePredictor(nn.Module):
    """Predicts L dynamic planes with parameters (a,b,c)"""
    def __init__(self, c_dim=32, num_planes=3):
        super().__init__()
        self.num_planes = num_planes
        
        # Simple PointNet for plane prediction
        self.pointnet = nn.Sequential(
            nn.Linear(3, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32)
        )
        
        # Plane parameter prediction
        self.plane_fc = nn.Linear(32, num_planes * 3)
        
        # Plane-specific feature extraction
        self.feature_fc = nn.Linear(32, num_planes * c_dim)
        
    def forward(self, points):
        batch_size = points.shape[0]
        
        # Extract features
        features = self.pointnet(points)
        
        # Global max pooling
        global_features = torch.max(features, dim=1)[0]
        
        # Predict plane parameters
        plane_params = self.plane_fc(global_features)
        plane_params = plane_params.view(batch_size, self.num_planes, 3)
        
        # Normalize to unit vectors
        plane_params = F.normalize(plane_params, p=2, dim=-1)
        
        # Extract plane-specific features
        plane_features = self.feature_fc(global_features)
        plane_features = plane_features.view(batch_size, self.num_planes, -1)
        
        return plane_params, plane_features

In [ ]:
class PlanarProjection(nn.Module):
    """Complex planar projection with basis change and normalization"""
    def __init__(self, plane_resolution=64):
        super().__init__()
        self.plane_resolution = plane_resolution
        
    def forward(self, points, features, plane_normals):
        batch_size, num_points, feature_dim = features.shape
        num_planes = plane_normals.shape[1]
        
        projected_features = []
        
        for i in range(num_planes):
            normal = plane_normals[:, i, :]
            
            # Project features onto this plane
            plane_features = self._project_to_plane(points, features, normal)
            projected_features.append(plane_features)
        
        return torch.stack(projected_features, dim=1) 
    
    def _project_to_plane(self, points, features, normal):
        # Proper orthogonal projection
        batch_size, num_points, feature_dim  = features.shape

        if normal.dim() == 3:
            normal = normal.squeeze(1)
            
        # Create orthonormal basis for plane
        # Arbitrary vector not parallele to normal
        arbitrary = torch.tensor([1.0, 0.0, 0.0], device=normal.device)
        
        # Check dot product for each batch item
        dots = torch.abs(torch.sum(normal * arbitrary.unsqueeze(0), dim=-1))
        need_alternate = dots > 0.9
        
        # Use alternate vector where needed
        alternate = torch.tensor([0.0, 1.0, 0.0], device=normal.device)
        arbitrary_batch = torch.where(
            need_alternate.unsqueeze(-1), 
            alternate.unsqueeze(0).expand_as(normal),
            arbitrary.unsqueeze(0).expand_as(normal)
        )
        
        # Gram-Schmidt to get orthonormal basis
        u1 = normal
        u2 = arbitrary.unsqueeze(0).expand_as(normal)
        u2 = u2 - torch.sum(u2 * u1, dim=-1, keepdim=True) * u1
        u2 = F.normalize(u2, dim=-1)

        # Project points onto plane basis
        coords_2d = torch.stack([
            torch.sum(points * u1.unsqueeze(1), dim=-1),
            torch.sum(points * u2.unsqueeze(1), dim=-1)
        ], dim=-1)
        
        # Normalize to [0, plane_resolution-1]
        coords_min = coords_2d.min(dim=1, keepdim=True)[0]
        coords_max = coords_2d.max(dim=1, keepdim=True)[0]
        coords_2d = (coords_2d - coords_min) / (coords_max - coords_min + 1e-8)
        coords_2d = coords_2d * (self.plane_resolution - 1)
        coords_2d = coords_2d.long().clamp(0, self.plane_resolution - 1)
            
        # Create feature map
        feature_map = torch.zeros(
            batch_size, feature_dim, self.plane_resolution, self.plane_resolution,
            device=points.device
        )
        
        # Scatter features
        coords_flat = coords_2d.view(-1, 2)                
        features_flat = features.reshape(-1, feature_dim)
        batch_indices = torch.arange(batch_size).repeat_interleave(num_points)

        for b in range(batch_size):
            mask = batch_indices == b
            coords_b = coords_flat[mask]
            features_b = features_flat[mask] 
            feature_map[b, :, coords_b[:, 1], coords_b[:, 0]] += features_b.T
        
        return feature_map

In [ ]:
class UNetBlock(nn.Module):
    """A standard U-Net block with two convolutional layers."""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.double_conv(x)

In [ ]:
class PlaneUNet(nn.Module):
    """U-Net for processing plane features"""
    def __init__(self, c_dim=32):
        super().__init__()
        
        # Encoder
        self.enc1 = UNetBlock(c_dim, 64)
        self.enc2 = UNetBlock(64, 128)
        self.enc3 = UNetBlock(128, 256)
        
        # Decoder  
        self.dec3 = UNetBlock(256, 128)
        self.dec2 = UNetBlock(128, 64)
        self.dec1 = UNetBlock(64, c_dim)
        
        self.pool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        
        # Decoder
        d3 = self.dec3(e3)
        d2 = self.dec2(self.upsample(d3) + e2) 
        d1 = self.dec1(self.upsample(d2) + e1)
        
        return d1

In [ ]:
class PositionalEncoding(nn.Module):
    """Positional encoding as used in NeRF and TransONet"""
    def __init__(self, L=10):
        super().__init__()
        self.L = L
        
    def forward(self, x):
        encoded = []
        for i in range(self.L):
            encoded.append(torch.sin(2**i * np.pi * x))
            encoded.append(torch.cos(2**i * np.pi * x))
        return torch.cat(encoded, dim=-1)

In [ ]:
class TransONetDecoder(nn.Module):
    """TransONet decoder with bilinear interpolation"""
    def __init__(self, c_dim=32, num_planes=3, dropout_rate=0.2):
        super().__init__()
        self.num_planes = num_planes
        
        # Positional encoding
        self.pos_encoding = PositionalEncoding(L=10)
        
        # Occupancy prediction network
        self.fc_layers = nn.ModuleList([
            nn.Linear(60 + num_planes * c_dim, c_dim),
            nn.Linear(c_dim, c_dim),
            nn.Linear(c_dim, c_dim),
            nn.Linear(c_dim, 1)
        ])

        self.dropout_layers = nn.ModuleList([
              nn.Dropout(dropout_rate) for _ in range(len(self.fc_layers) - 1)  # No dropout before final layer
        ])
        
    def forward(self, query_points, plane_features, plane_normals):
        batch_size, num_queries, _ = query_points.shape
        
        # Apply positional encoding
        pos_encoded = self.pos_encoding(query_points)
        
        # Sample features from planes using bilinear interpolation
        sampled_features = []
        for i in range(self.num_planes):
            # Project query points to plane i
            plane_coords = self._project_points_to_plane(query_points, plane_normals[:, i, :])
            
            # Sample features using bilinear interpolation
            sampled = F.grid_sample(
                plane_features[:, i], 
                plane_coords.unsqueeze(2),
                mode='bilinear', padding_mode='border', align_corners=True
            ).squeeze(-1).transpose(1, 2) 
            
            sampled_features.append(sampled)
        
        # Concatenate all plane features
        plane_features_concat = torch.cat(sampled_features, dim=-1) 
        
        # Combine with positional encoding
        features = torch.cat([pos_encoded, plane_features_concat], dim=-1)
        
        # Predict occupancy
        x = features
        for fc in self.fc_layers[:-1]:
            x = F.relu(fc(x))
            x = self.dropout_layers[i](x)
            
        raw_logits = self.fc_layers[-1](x)
        occupancy = raw_logits.squeeze(-1)
        
        return occupancy
    
    def _project_points_to_plane(self, points, normal):
        # Simplified projection
        coords_2d = points[:, :, [0, 1]] 
        return coords_2d

In [ ]:
class TransONet(nn.Module):
    """Complete TransONet architecture as required by Project 5"""
    def __init__(self, c_dim=32, num_planes=3, plane_resolution=64, dropout_rate=0.1): 
        super().__init__()
        
        # Components
        self.encoder = ResNetPointNet(c_dim=c_dim, dropout_rate=dropout_rate)
        self.plane_predictor = DynamicPlanePredictor(c_dim=c_dim, num_planes=num_planes)
        self.planar_projection = PlanarProjection(plane_resolution=plane_resolution)
        self.unet = PlaneUNet(c_dim=c_dim)
        self.decoder = TransONetDecoder(c_dim=c_dim, num_planes=num_planes, dropout_rate=dropout_rate*2)
        self.similarity_loss = SimilarityLoss()
        
        print(f"TransONet initialized with {self.count_parameters():,} parameters")
        
    def forward(self, point_cloud, query_points, return_plane_params=False):
        batch_size = point_cloud.shape[0]
        
        # Encode point cloud
        point_features = self.encoder(point_cloud)
        
        # Predict dynamic planes
        plane_normals, plane_features = self.plane_predictor(point_cloud)
        
        # Expand plane features and add to point features
        expanded_plane_features = plane_features.unsqueeze(2).expand(-1, -1, point_features.shape[1], -1)
        point_features_expanded = point_features.unsqueeze(1).expand(-1, plane_normals.shape[1], -1, -1)
        combined_features = point_features_expanded + expanded_plane_features
        
        # Project features onto dynamic planes
        projected_features = []
        for i in range(plane_normals.shape[1]):
            proj_feat = self.planar_projection._project_to_plane(
                point_cloud, combined_features[:, i, :, :], plane_normals[:, i, :]
            )
            projected_features.append(proj_feat)
        
        projected_features = torch.stack(projected_features, dim=1) 
        
        # Process with U-Net
        processed_features = []
        for i in range(plane_normals.shape[1]):
            processed = self.unet(projected_features[:, i, :, :, :])
            processed_features.append(processed)
        
        processed_features = torch.stack(processed_features, dim=1) 
        
        # Decode occupancy
        occupancy = self.decoder(query_points, processed_features, plane_normals)
        
        if return_plane_params:
            return occupancy, plane_normals
        else:
            return occupancy
    
    def compute_loss(self, occupancy_pred, occupancy_true, plane_normals, similarity_weight=0.1):
        # Main occupancy loss
        pos_ratio = occupancy_true.mean()
        pos_weight = (1.0 - pos_ratio) / pos_ratio if pos_ratio > 0 else torch.tensor(100.0)
        pos_weight_tensor = torch.tensor([pos_weight], device=occupancy_true.device)
    
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
        occupancy_loss = criterion(occupancy_pred, occupancy_true)
        
        # Similarity loss for plane diversity
        sim_loss = self.similarity_loss(plane_normals)
        
        total_loss = occupancy_loss + similarity_weight * sim_loss
        return total_loss, occupancy_loss, sim_loss
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def compute_similarity_loss(self, plane_normals):
        return self.similarity_loss(plane_normals)

In [ ]:
class SimilarityLoss(nn.Module):
    """Similarity loss to ensure plane diversity"""
    def __init__(self):
        super().__init__()
        
    def forward(self, plane_normals):
        batch_size, num_planes, _ = plane_normals.shape
        
        # Compute pairwise dot products
        similarity = torch.bmm(plane_normals, plane_normals.transpose(1, 2))
        
        # Remove diagonal (self-similarity)
        mask = torch.eye(num_planes, device=plane_normals.device).unsqueeze(0)
        similarity = similarity * (1 - mask)
        
        # Penalize high similarity (want diverse planes)
        loss = torch.mean(torch.abs(similarity))
        return loss

# Train

In [ ]:
class Trainer:
    """Training class for Transonet"""

    def __init__(self, model, optimizer, device, loss_fn=None, 
             patience=10, min_delta=1e-4, scheduler_type='ReduceLROnPlateau', 
             scheduler_patience=5, scheduler_factor=0.5, checkpoint_name='best_model.pth'): 
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.criterion = loss_fn or nn.BCEWithLogitsLoss()
        self.checkpoint_path = os.path.join(MODEL_SAVE_PATH, checkpoint_name)

        # Training metrics
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

        # Early stopping parameters
        self.patience = patience
        self.min_delta = min_delta
        self.best_val_loss = float('inf')
        self.epochs_without_improvement = 0
        self.early_stopped = False
        self.best_model_state = None

        # Learning rate scheduler step
        self.scheduler_type = scheduler_type
        if scheduler_type == 'ReduceLROnPlateau':
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=scheduler_factor, 
                patience=scheduler_patience, verbose=True, min_lr=1e-7
            )
        elif scheduler_type == 'StepLR':
            self.scheduler = optim.lr_scheduler.StepLR(
                optimizer, step_size=scheduler_patience, gamma=scheduler_factor
            )
        elif scheduler_type == 'CosineAnnealingLR':
            self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=50, eta_min=1e-7
            )
        else:
            self.scheduler = None
            print(f"Warning: Unknown scheduler type {scheduler_type}, using no scheduler")

    def validate(self, val_loader):
        """Validate the model"""
        self.model.eval()
        total_loss = 0
        total_samples = 0
        all_predictions = []
        all_targets = []

        with torch.no_grad():
            for batch in val_loader:
                point_cloud = batch['point_cloud'].to(self.device)
                sample_points = batch['sample_points'].to(self.device)
                occupancy = batch['occupancy'].to(self.device)

                # Handle TransONet dual output
                occupancy_logits, plane_params = self.model(point_cloud, sample_points, return_plane_params=True)

                # Calculate pos_weight
                pos_ratio = occupancy.mean()
                pos_weight = (1.0 - pos_ratio) / pos_ratio if pos_ratio > 0 else torch.tensor(100.0)
                pos_weight_tensor = torch.tensor([pos_weight], device=self.device)

                # Use BCE with pos_weight
                criterion_with_posweight = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
                occ_loss = criterion_with_posweight(occupancy_logits, occupancy)

                # Add similarity loss
                sim_loss = self.model.compute_similarity_loss(plane_params)
                loss = occ_loss + 0.1 * sim_loss
            
                # Statistics
                total_loss += loss.item() * point_cloud.size(0)
                total_samples += point_cloud.size(0)

                # Collect predictions and targets for sklearn metrics
                predictions = torch.sigmoid(occupancy_logits) > 0.5
                all_predictions.append(predictions.cpu().numpy().flatten())
                all_targets.append(occupancy.cpu().numpy().flatten())
            

        # Convert to numpy arrays for sklearn
        all_predictions = np.concatenate(all_predictions)
        all_targets = np.concatenate(all_targets)
        
        # Use sklearn metrics
        accuracy = accuracy_score(all_targets, all_predictions)
    
        avg_loss = total_loss / total_samples
        return avg_loss, accuracy

    def check_early_stopping(self, val_loss):
        """Check if training should stop early and update best model"""
        improved = False
        
        if val_loss < self.best_val_loss - self.min_delta:
            self.best_val_loss = val_loss
            self.epochs_without_improvement = 0
            self.best_model_state = self.model.state_dict().copy()
            improved = True
            print(f"✓ New best validation loss: {val_loss:.6f}")
        else:
            self.epochs_without_improvement += 1
            
        if self.epochs_without_improvement >= self.patience:
            self.early_stopped = True
            print(f"🛑 Early stopping triggered! No improvement for {self.patience} epochs")
            if self.best_model_state is not None:
                self.model.load_state_dict(self.best_model_state)
                print("📁 Restored best model weights")
        
        return improved

    def train(self, train_loader, val_loader, num_epochs):
        print("Starting training...")
        print(f"Model parameters: {count_parameters(self.model):,}")
        print(f"Early stopping: patience={self.patience}, min_delta={self.min_delta}")
        print(f"LR Scheduler: {self.scheduler_type}")

        best_val_loss = float('inf')

        # Progress bar for epochs
        epoch_pbar = tqdm(range(num_epochs), desc="Training Progress")

        for epoch in epoch_pbar:
            start_time = time.time()

            # Train with batch progress bar
            train_loss, train_acc = self.train_epoch_with_progress(train_loader, epoch)
            self.train_losses.append(train_loss)
            self.train_accuracies.append(train_acc)

            # Validate
            val_loss, val_acc = self.validate(val_loader)
            self.val_losses.append(val_loss)
            self.val_accuracies.append(val_acc)

            # Early stopping
            improved = self.check_early_stopping(val_loss)

            if self.scheduler:
                if self.scheduler_type == 'ReduceLROnPlateau':
                    self.scheduler.step(val_loss)
                else:
                    self.scheduler.step()
                    
                current_lr = self.optimizer.param_groups[0]['lr']
                if improved:
                    print(f"📈 Current LR: {current_lr:.8f}")

            # Real-time plotting
            self.plot_live_progress()

            # Update progress bar
            epoch_pbar.set_postfix({
                'Train_Loss': f'{train_loss:.4f}',
                'Val_Loss': f'{val_loss:.4f}',
                'Train_Acc': f'{train_acc:.4f}',
                'Val_Acc': f'{val_acc:.4f}'
            })

            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                save_checkpoint(self.model, self.optimizer, epoch, val_loss,
                            self.checkpoint_path)

            if self.early_stopped:
                print(f"🏁 Training stopped early at epoch {epoch+1}")
                break

        # Training completion summary
        if not self.early_stopped:
            print(f"✅ Training completed normally after {num_epochs} epochs")
        
        final_lr = self.optimizer.param_groups[0]['lr']
        print(f"Final LR: {final_lr:.8f}")
        print(f"Best validation loss: {self.best_val_loss:.6f}")

    def plot_live_progress(self):
        clear_output(wait=True)
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))

        # Loss plot
        axes[0].plot(self.train_losses, label='Train Loss', color='blue')
        if self.val_losses:
            val_epochs = list(range(len(self.val_losses)))
            axes[0].plot(val_epochs, self.val_losses, label='Val Loss', color='red')

            if hasattr(self, 'best_val_loss') and self.best_val_loss != float('inf'):
                axes[0].axhline(y=self.best_val_loss, color='green', linestyle='--', alpha=0.7, label=f'Best Val: {self.best_val_loss:.4f}')
        
        axes[0].set_title('Loss Progress')
        axes[0].legend()
        axes[0].grid(True)

        # Accuracy plot
        axes[1].plot(self.train_accuracies, label='Train Acc', color='blue')
        if self.val_accuracies:
            val_epochs = list(range(len(self.val_accuracies)))
            axes[1].plot(val_epochs, self.val_accuracies, label='Val Acc', color='red')
        axes[1].set_title('Accuracy Progress')
        axes[1].legend()
        axes[1].grid(True)

        plt.tight_layout()
        plt.show()

    def train_epoch_with_progress(self, train_loader, epoch):
        self.model.train()
        total_loss = 0
        total_samples = 0
        correct_predictions = 0

        batch_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

        for batch_idx, batch in enumerate(batch_pbar):
            point_cloud = batch['point_cloud'].to(self.device)
            sample_points = batch['sample_points'].to(self.device)
            occupancy = batch['occupancy'].to(self.device)

            self.optimizer.zero_grad()
            occupancy_logits, plane_normals = self.model(point_cloud, sample_points, return_plane_params=True)
            loss, occ_loss, sim_loss = self.model.compute_loss(
                occupancy_logits, occupancy, plane_normals
            )
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item() * point_cloud.size(0)
            total_samples += point_cloud.size(0)

            predictions = torch.sigmoid(occupancy_logits) > 0.5
            correct_predictions += (predictions == occupancy).sum().item()

            batch_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / total_samples
        accuracy = correct_predictions / (total_samples * SAMPLE_POINTS)
        return avg_loss, accuracy

## Model Creation

In [ ]:
def create_transonet_model_and_optimizer():
    """Create TransONet model and optimizer"""
    model = TransONet(
        c_dim=32,
        num_planes=3,
        plane_resolution=64,
        dropout_rate=DROPOUT_RATE
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    print(f"TransONet created with {count_parameters(model):,} parameters")
    return model, optimizer

In [ ]:
print("\n3. MODEL CREATION")
print("-" * 40)

model, optimizer = create_transonet_model_and_optimizer()
model.decoder.fc_layers[-1].bias.data.fill_(0.02) # Bias to avoid collapse to 0 of predictions 
print("Model created")

In [ ]:
print("\n4. TRAINING")
print("-" * 60)

trainer = Trainer(model, optimizer, device,
                patience=15,              # Stop after 15 epochs without improvement
                min_delta=1e-5,          # Minimum improvement threshold
                scheduler_type='ReduceLROnPlateau',  # Options: 'ReduceLROnPlateau', 'StepLR', 'CosineAnnealingLR'
                scheduler_patience=7,     
                scheduler_factor=0.5,      # Reduce LR by half
                checkpoint_name='best_model.pth'
                )
trainer.train(train_loader, val_loader, NUM_EPOCHS)

In [ ]:
def plot_training_curves(trainer, filename='training_curves.png'): 
    """Plot training and validation curves"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Loss curves
    axes[0].plot(trainer.train_losses, label='Train Loss')
    if trainer.val_losses:
        val_epochs = list(range(len(trainer.val_losses)))
        axes[0].plot(val_epochs, trainer.val_losses, label='Validation Loss')

    if hasattr(trainer, 'best_val_loss') and trainer.best_val_loss != float('inf'):
        axes[0].axhline(y=trainer.best_val_loss, color='green', linestyle='--', alpha=0.7,
                        label=f'Best Val Loss: {trainer.best_val_loss:.4f}')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True)

    # Accuracy curves
    axes[1].plot(trainer.train_accuracies, label='Train Accuracy')
    if trainer.val_accuracies:
        val_epochs = list(range(len(trainer.val_losses)))
        axes[1].plot(val_epochs, trainer.val_accuracies, label='Validation Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_PATH, filename), dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
print("\n5. TRAINING CURVES")
print("-" * 40)

plot_training_curves(trainer, filename='training_curves.png')
print("\nPlotting original training curves\n")

# Test

In [ ]:
class TransONetEvaluator:
    """Proper evaluator for TransONet occupancy predictions"""
    
    def __init__(self, model, device):
        self.model = model
        self.device = device
    
    def evaluate_occupancy_metrics(self, test_loader):
        """Evaluate occupancy prediction metrics"""
        self.model.eval()
        
        all_predictions = []
        all_targets = []
        inference_times = []
        
        with torch.no_grad():
            for batch in test_loader:
                point_cloud = batch['point_cloud'].to(self.device)
                sample_points = batch['sample_points'].to(self.device) 
                occupancy = batch['occupancy'].to(self.device)
                
                # Measure inference time
                start_time = time.time()
                occupancy_logits = self.model(point_cloud, sample_points)
                inference_time = time.time() - start_time
                inference_times.append(inference_time)
                
                # Convert logits to predictions
                predictions = torch.sigmoid(occupancy_logits) > 0.5
                
                all_predictions.append(predictions.cpu().numpy().flatten())
                all_targets.append(occupancy.cpu().numpy().flatten())
        
        # Flatten arrays
        all_predictions = np.concatenate(all_predictions)
        all_targets = np.concatenate(all_targets)
        
        # Core occupancy metrics
        accuracy = accuracy_score(all_targets, all_predictions)
        
        # Handle edge cases with sklearn's zero_division parameter
        if all_predictions.sum() > 0:
            precision = precision_score(all_targets, all_predictions, zero_division=0)
        else:
            precision = 0.0

        if all_targets.sum() > 0:
            recall = recall_score(all_targets, all_predictions, zero_division=0)
            f1 = f1_score(all_targets, all_predictions, zero_division=0)
        else:
            recall = f1 = 0.0
        
        # Volumetric IoU (more appropriate for occupancy)
        intersection = ((all_predictions == 1) & (all_targets == 1)).sum()
        union = ((all_predictions == 1) | (all_targets == 1)).sum()
        volumetric_iou = intersection / union if union > 0 else 0
        
        return {
            'occupancy_accuracy': accuracy,
            'occupancy_precision': precision,
            'occupancy_recall': recall,
            'occupancy_f1': f1,
            'volumetric_iou': volumetric_iou,
            'avg_inference_time': np.mean(inference_times),
            'std_inference_time': np.std(inference_times),
            'throughput': len(test_loader.dataset) / sum(inference_times)
        }
    
    def evaluate_mesh_quality(self, test_loader, resolution=64):
        """Evaluate mesh quality if ground truth meshes available"""
        # Only run if ground truth meshes are available
        dataset = test_loader.dataset
        if not hasattr(dataset, 'has_ground_truth') or not dataset.has_ground_truth:
            return {}
        
        self.model.eval()
        mesh_metrics = []
        
        with torch.no_grad():
            for i, batch in enumerate(test_loader):
                if i >= 10:  # Limit for computational efficiency
                    break
                    
                point_cloud = batch['point_cloud'].to(self.device)
                file_id = batch.get('file_id', [None])[0]
                
                if file_id is None:
                    continue
                
                # Generate occupancy field
                occupancy_field = self._generate_occupancy_field(point_cloud, resolution)
                
                # Extract mesh using marching cubes (if trimesh available)
                try:
                    import trimesh
                    from skimage.measure import marching_cubes
                    
                    verts, faces, _, _ = marching_cubes(occupancy_field, level=0.5)
                    predicted_mesh = trimesh.Trimesh(vertices=verts, faces=faces)
                    
                    # Load ground truth mesh
                    gt_mesh_path = dataset.get_ground_truth_mesh_path(file_id)
                    if gt_mesh_path and os.path.exists(gt_mesh_path):
                        gt_mesh = trimesh.load(gt_mesh_path)
                        
                        # Compute mesh metrics
                        metrics = self._compute_mesh_metrics(predicted_mesh, gt_mesh)
                        mesh_metrics.append(metrics)
                        
                except ImportError:
                    print("Trimesh not available for mesh evaluation")
                    break
        
        if mesh_metrics:
            return {
                'mesh_hausdorff_distance': np.mean([m['hausdorff'] for m in mesh_metrics]),
                'mesh_chamfer_distance': np.mean([m['chamfer'] for m in mesh_metrics]),
                'mesh_volume_error': np.mean([m['volume_error'] for m in mesh_metrics])
            }
        return {}
    
    def _generate_occupancy_field(self, point_cloud, resolution):
        """Generate 3D occupancy field for mesh extraction"""
        # Create regular grid
        coords = np.linspace(-1, 1, resolution)
        grid_x, grid_y, grid_z = np.meshgrid(coords, coords, coords, indexing='ij')
        query_points = np.stack([grid_x, grid_y, grid_z], axis=-1).reshape(-1, 3)
        
        # Process in batches to avoid memory issues
        batch_size = 10000
        occupancy_values = []
        
        for i in range(0, len(query_points), batch_size):
            point_cloud_single = point_cloud[:1]  
            batch_points = torch.FloatTensor(query_points[i:i+batch_size]).unsqueeze(0).to(self.device)
            logits = self.model(point_cloud_single, batch_points)
            occupancy = torch.sigmoid(logits) > 0.5
            occupancy_values.append(occupancy.cpu().numpy().flatten())
        
        occupancy_field = np.concatenate(occupancy_values).reshape(resolution, resolution, resolution)
        return occupancy_field.astype(float)
    
    def _compute_mesh_metrics(self, pred_mesh, gt_mesh):
        """Compute mesh-to-mesh distance metrics"""
        if pred_mesh.volume < 0:
            pred_mesh.invert()
        
        # Normalize both to unit scale and center
        pred_mesh.apply_scale(1.0 / pred_mesh.scale)
        gt_mesh.apply_scale(1.0 / gt_mesh.scale)
        
        pred_mesh.vertices -= pred_mesh.centroid
        gt_mesh.vertices -= gt_mesh.centroid
    
        try:
            pred_points, _ = pred_mesh.sample(1000)
            gt_points, _ = gt_mesh.sample(1000)
            
            if pred_points.ndim == 1:
                pred_points = pred_points.reshape(1, -1)
            if gt_points.ndim == 1:
                gt_points = gt_points.reshape(1, -1)
                
        except:
            pred_points = pred_mesh.vertices[:1000]
            gt_points = gt_mesh.vertices[:1000]
            
        # Compute distances
        from scipy.spatial.distance import cdist
        
        # Hausdorff distance
        dist_pred_to_gt = cdist(pred_points, gt_points).min(axis=1)
        dist_gt_to_pred = cdist(gt_points, pred_points).min(axis=1)
        hausdorff = max(dist_pred_to_gt.max(), dist_gt_to_pred.max())
        
        # Chamfer distance
        chamfer = dist_pred_to_gt.mean() + dist_gt_to_pred.mean()
        
        # Volume error
        volume_error = abs(pred_mesh.volume - gt_mesh.volume) / gt_mesh.volume
        
        return {
            'hausdorff': hausdorff,
            'chamfer': chamfer,
            'volume_error': volume_error
        }
    
    def print_metrics(self, metrics):
        """Print metrics in formatted way"""
        print("=" * 60)
        print("TRANSONET OCCUPANCY NETWORK EVALUATION")
        print("=" * 60)
        
        print(f"\nOCCUPANCY PREDICTION METRICS:")
        print(f"  Accuracy:          {metrics['occupancy_accuracy']:.4f}")
        print(f"  Precision:         {metrics['occupancy_precision']:.4f}")
        print(f"  Recall:            {metrics['occupancy_recall']:.4f}")
        print(f"  F1-Score:          {metrics['occupancy_f1']:.4f}")
        print(f"  Volumetric IoU:    {metrics['volumetric_iou']:.4f}")
        
        print(f"\n⏱️ TIMING METRICS:")
        print(f"  Avg Inference:     {metrics['avg_inference_time']:.4f}s")
        print(f"  Throughput:        {metrics['throughput']:.2f} samples/s")
        
        # Print mesh metrics if available
        mesh_keys = [k for k in metrics.keys() if k.startswith('mesh_')]
        if mesh_keys:
            print(f"\n🎭 MESH QUALITY METRICS:")
            for key in mesh_keys:
                print(f"  {key.replace('mesh_', '').title()}: {metrics[key]:.4f}")


# Usage example:
def run_proper_evaluation():
    """Run evaluation with proper metrics for TransONet"""
    
    # Create evaluator
    evaluator = TransONetEvaluator(model, device)
    
    # Evaluate occupancy metrics (core evaluation)
    occupancy_metrics = evaluator.evaluate_occupancy_metrics(val_loader)
    
    # Evaluate mesh quality (if ground truth available)
    mesh_metrics = evaluator.evaluate_mesh_quality(val_loader)
    
    # Combine metrics
    all_metrics = {**occupancy_metrics, **mesh_metrics}
    
    # Print results
    evaluator.print_metrics(all_metrics)
    
    return all_metrics

## Evaluation

In [ ]:
# Test calibration after training
def test_current_calibration():
    model.eval()
    with torch.no_grad():
        batch = next(iter(val_loader))
        point_cloud = batch['point_cloud'].to(device)
        sample_points = batch['sample_points'].to(device)
        
        logits = model(point_cloud, sample_points)
        preds = torch.sigmoid(logits) > 0.5
        pred_ratio = preds.float().mean().item()
        
        print(f"Current prediction ratio: {pred_ratio:.4f}")
        print(f"Target should be: ~0.01")
        
        return pred_ratio

test_current_calibration()

In [ ]:
print("\n6. OCCUPANCY EVALUATION")
print("-" * 40)
print("Running TransONet evaluation...")
comprehensive_metrics = run_proper_evaluation()

## Speed Benchmark

In [ ]:
# 7. SPEED BENCHMARK
print("\n7. SPEED BENCHMARK")
print("-" * 40)

evaluator = TransONetEvaluator(model, device)
metrics = evaluator.evaluate_occupancy_metrics(val_loader)

print(f"  avg_inference_time: {metrics['avg_inference_time']:.6f}s")
print(f"  std_inference_time: {metrics['std_inference_time']:.6f}s") 
print(f"  throughput: {metrics['throughput']:.2f} samples/s")

In [ ]:
# Simplified version for consistent results
def analyze_learned_planes_summary(model, val_loader, device):
    """Analyze planes for comparison purposes"""
    model.eval()
    plane_orientations = []
    
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i >= 10:  # Sufficient for analysis
                break
            point_cloud = batch['point_cloud'].to(device)
            sample_points = batch['sample_points'].to(device)
            _, plane_params = model(point_cloud, sample_points, return_plane_params=True)
            plane_orientations.append(plane_params.cpu().numpy())
    
    plane_orientations = np.concatenate(plane_orientations, axis=0)
    
    # Return summary statistics instead of detailed output
    plane_stats = []
    for plane_idx in range(plane_orientations.shape[1]):
        orientations = plane_orientations[:, plane_idx, :]
        avg_orientation = np.mean(orientations, axis=0)
        std_dev = np.std(orientations, axis=0)
        consistency = 1.0 - np.mean(std_dev)
        plane_stats.append({
            'plane_id': plane_idx + 1,
            'consistency': consistency,
            'avg_normal': avg_orientation.tolist()
        })
    
    return plane_stats

plane_analysis = analyze_learned_planes_summary(model, val_loader, device)

## Summary

In [ ]:
print("\n8. SUMMARY")
print("-" * 40)
print(f"Accuracy: {comprehensive_metrics['occupancy_accuracy']:.4f}")
print(f"Precision: {comprehensive_metrics['occupancy_precision']:.4f}")
print(f"Recall: {comprehensive_metrics['occupancy_recall']:.4f}")
print(f"F1-Score: {comprehensive_metrics['occupancy_f1']:.4f}")
print(f"Volumetric IoU: {comprehensive_metrics['volumetric_iou']:.4f}")

In [ ]:
print("="*80)
print("TRANSONET FINAL RESULTS - PROJECT 5 EVALUATION")
print("="*80)

# CORE OCCUPANCY METRICS (Primary Project 5 objective)
evaluator = TransONetEvaluator(model, device)
occupancy_metrics = evaluator.evaluate_occupancy_metrics(val_loader)

# MESH QUALITY METRICS
mesh_metrics = evaluator.evaluate_mesh_quality(val_loader)

# ARCHITECTURE ANALYSIS
model.eval()
with torch.no_grad():
    sample_batch = next(iter(val_loader))
    point_cloud = sample_batch['point_cloud'].to(device)
    sample_points = sample_batch['sample_points'].to(device)
    
    logits = model(point_cloud, sample_points)
    predictions = torch.sigmoid(logits)
    prediction_ratio = (predictions > 0.5).float().mean().item()

# LEARNED PLANE ANALYSIS (TransONet specific)
def analyze_planes_summary(model, val_loader, device):
    model.eval()
    plane_orientations = []
    
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i >= 5:  # Quick analysis
                break
            point_cloud = batch['point_cloud'].to(device)
            sample_points = batch['sample_points'].to(device)
            _, plane_params = model(point_cloud, sample_points, return_plane_params=True)
            plane_orientations.append(plane_params.cpu().numpy())
    
    plane_orientations = np.concatenate(plane_orientations, axis=0)
    
    # Analyze plane consistency
    plane_consistency = []
    for plane_idx in range(plane_orientations.shape[1]):
        orientations = plane_orientations[:, plane_idx, :]
        std_dev = np.std(orientations, axis=0)
        consistency = 1.0 - np.mean(std_dev)  # Higher = more consistent
        plane_consistency.append(consistency)
    
    return np.mean(plane_consistency)

plane_consistency = analyze_planes_summary(model, val_loader, device)

# COMPILE RESULTS DICTIONARY FOR COMPARISON
results = {
    # APPROACH IDENTIFICATION
    'approach_name': 'TransONet',
    'dataset': DATASET_NAME,
    'model_parameters': count_parameters(model),
    
    # CORE OCCUPANCY METRICS (Project 5 primary)
    'occupancy_accuracy': occupancy_metrics['occupancy_accuracy'],
    'occupancy_precision': occupancy_metrics['occupancy_precision'],
    'occupancy_recall': occupancy_metrics['occupancy_recall'],
    'occupancy_f1': occupancy_metrics['occupancy_f1'],
    'volumetric_iou': occupancy_metrics['volumetric_iou'],

    # MESH QUALITY METRICS
    'mesh_hausdorff_distance': mesh_metrics.get('mesh_hausdorff_distance', None),
    'mesh_chamfer_distance': mesh_metrics.get('mesh_chamfer_distance', None),
    'mesh_volume_error': mesh_metrics.get('mesh_volume_error', None),
    
    # EFFICIENCY METRICS
    'avg_inference_time': occupancy_metrics['avg_inference_time'],
    'throughput': occupancy_metrics['throughput'],
    
    # MODEL-SPECIFIC METRICS
    'prediction_ratio': prediction_ratio,
    'plane_consistency': plane_consistency,
    
    # TRAINING CONFIGURATION
    'batch_size': BATCH_SIZE,
    'epochs_trained': NUM_EPOCHS,
    'learning_rate': LEARNING_RATE,
    'point_cloud_size': POINT_CLOUD_SIZE,
    'sample_points': SAMPLE_POINTS,
}

# DISPLAY RESULTS IN COMPARISON-READY FORMAT
print(f"\nRESULTS SUMMARY - {results['approach_name'].upper()}")
print("-" * 60)

print(f"\nCORE METRICS (Project 5 Primary Objectives):")
print(f"  Occupancy Accuracy:    {results['occupancy_accuracy']:.4f}")
print(f"  Occupancy Precision:   {results['occupancy_precision']:.4f}")
print(f"  Occupancy Recall:      {results['occupancy_recall']:.4f}")
print(f"  Occupancy F1-Score:    {results['occupancy_f1']:.4f}")
print(f"  Volumetric IoU:        {results['volumetric_iou']:.4f}")

print(f"\nMESH QUALITY METRICS:")
if results['mesh_hausdorff_distance'] is not None:
    print(f"  Hausdorff Distance:    {results['mesh_hausdorff_distance']:.4f}")
    print(f"  Chamfer Distance:      {results['mesh_chamfer_distance']:.4f}")
    print(f"  Volume Error:          {results['mesh_volume_error']:.4f}")
else:
    print(f"  No ground truth meshes available")

print(f"\nEFFICIENCY METRICS:")
print(f"  Inference Time:        {results['avg_inference_time']:.4f}s")
print(f"  Throughput:            {results['throughput']:.2f} samples/s")
print(f"  Model Parameters:      {results['model_parameters']:,}")

print(f"\nARCHITECTURE-SPECIFIC:")
print(f"  Prediction Ratio:      {results['prediction_ratio']:.4f}")
print(f"  Plane Consistency:     {results['plane_consistency']:.4f}")

print(f"\nCONFIGURATION:")
print(f"  Dataset:               {results['dataset']}")
print(f"  Epochs:                {results['epochs_trained']}")
print(f"  Batch Size:            {results['batch_size']}")
print(f"  Learning Rate:         {results['learning_rate']}")

# SAVE RESULTS FOR COMPARISON
import json
results_file = f"{RESULTS_PATH}/transonet_results.json"
with open(results_file, 'w') as f:
    # Convert numpy types to native Python for JSON serialization
    json_results = {k: float(v) if isinstance(v, (np.float32, np.float64)) else v 
                   for k, v in results.items()}
    json.dump(json_results, f, indent=2)

print(f"\n Results saved to: {results_file}")
print(f" Ready for approach comparison!")

# QUICK PERFORMANCE ASSESSMENT
def assess_performance(results):
    """Quick assessment for Project 5 objectives"""
    score = 0
    assessment = []
    
    # Occupancy prediction quality (main objective)
    if results['occupancy_f1'] > 0.15:
        score += 3
        assessment.append("✅ F1-Score: Good")
    elif results['occupancy_f1'] > 0.10:
        score += 2
        assessment.append("⚠️ F1-Score: Moderate")
    else:
        score += 1
        assessment.append("❌ F1-Score: Needs improvement")
    
    if results['volumetric_iou'] > 0.08:
        score += 2
        assessment.append("✅ IoU: Good")
    elif results['volumetric_iou'] > 0.05:
        score += 1
        assessment.append("⚠️ IoU: Moderate")
    else:
        assessment.append("❌ IoU: Needs improvement")
    
    # Efficiency
    if results['avg_inference_time'] < 0.15:
        score += 1
        assessment.append("✅ Fast inference")
    
    return score, assessment

score, assessment = assess_performance(results)
print(f"\nPERFORMANCE ASSESSMENT (Score: {score}/6):")
for item in assessment:
    print(f"  {item}")

print("\n" + "="*80)
print("EVALUATION COMPLETE - READY FOR COMPARISON WITH OTHER APPROACHES")
print("="*80)

## Extra: Pruning of the model and fine-tuning

In [ ]:
print("--- Loading best pre-trained model for pruning ---")
prune_model, prune_optimizer = create_transonet_model_and_optimizer()
checkpoint_path = os.path.join(MODEL_SAVE_PATH, 'best_model.pth')
load_checkpoint(prune_model, prune_optimizer, checkpoint_path)
print(f"Loaded checkpoint. Parameters before pruning: {count_parameters(prune_model):,}")

In [ ]:
# --- Apply Pruning ---
print(f"\n--- Applying L1 unstructured pruning with ratio: {PRUNING_RATIO} ---")
for name, module in prune_model.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        prune.l1_unstructured(module, name='weight', amount=PRUNING_RATIO)

In [ ]:
# --- Fine-Tune the Pruned Model ---
print(f"\n--- Fine-tuning the pruned model for up to {FINETUNE_EPOCHS} epochs ---")
finetune_optimizer = optim.Adam(prune_model.parameters(), lr=FINETUNE_LR)

In [ ]:
# NEW Trainer instance for fine-tuning with its own early stopping and scheduler
finetune_trainer = Trainer(
    model=prune_model,
    optimizer=finetune_optimizer,
    device=device,
    patience=FINE_TUNING_PATIENCE,
    min_delta=1e-5,
    scheduler_type='ReduceLROnPlateau',
    scheduler_patience=3,
    scheduler_factor=0.5,
    checkpoint_name='fine_tuned_model.pth'
)
finetune_trainer.train(train_loader, val_loader, FINETUNE_EPOCHS)

In [ ]:
print("\n--- Plotting fine-tuning curves ---")
plot_training_curves(finetune_trainer, filename='finetuned_training_curves.png') 

In [ ]:
print("\n--- Making pruning permanent on the best fine-tuned model ---")
# Use the model from the trainer, which holds the best state due to early stopping
best_finetuned_model = finetune_trainer.model 
for name, module in best_finetuned_model.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        if prune.is_pruned(module):
            prune.remove(module, 'weight')
print("Pruning made permanent.")

print("--- Saving the cleaned, permanent model ---")
save_checkpoint(
    model=best_finetuned_model, 
    optimizer=None, # Optimizer state is not needed for final eval
    epoch=finetune_trainer.best_val_loss, # Storing the best loss in the 'epoch' field for reference
    loss=finetune_trainer.best_val_loss,
    path=finetune_trainer.checkpoint_path
)

In [ ]:
print("="*80)
print("FINAL EVALUATION OF PRUNED & FINE-TUNED MODEL")
print("="*80)

# --- Load the best FINE-TUNED model ---
# Create a fresh model instance and optimizer to load into
final_model, final_optimizer = create_transonet_model_and_optimizer()
finetuned_checkpoint_path = os.path.join(MODEL_SAVE_PATH, 'fine_tuned_model.pth')

if not os.path.exists(finetuned_checkpoint_path):
    raise FileNotFoundError(f"Checkpoint not found at {finetuned_checkpoint_path}. Please ensure the fine-tuning step ran and saved the model.")
        
load_checkpoint(final_model, final_optimizer, finetuned_checkpoint_path)
print(f"Successfully loaded: {finetuned_checkpoint_path}")
print("Model state is permanent and ready for evaluation.")

# --- Evaluate ALL Metrics ---
print("\n--- Calculating all performance metrics... ---")
evaluator = TransONetEvaluator(final_model, device)

# Core Occupancy and Efficiency Metrics
occupancy_metrics = evaluator.evaluate_occupancy_metrics(val_loader)

# Mesh Quality Metrics (if applicable)
mesh_metrics = evaluator.evaluate_mesh_quality(val_loader)

# Architecture-Specific Metrics
final_model.eval()
with torch.no_grad():
    sample_batch = next(iter(val_loader))
    point_cloud = sample_batch['point_cloud'].to(device)
    sample_points = sample_batch['sample_points'].to(device)
    
    logits, plane_params = final_model(point_cloud, sample_points, return_plane_params=True)
    predictions = torch.sigmoid(logits)
    prediction_ratio = (predictions > 0.5).float().mean().item()
    
# Plane Consistency
plane_consistency = analyze_planes_summary(final_model, val_loader, device)


# --- Compile Comprehensive Results Dictionary ---
pruned_results = {
    # APPROACH IDENTIFICATION
    'approach_name': f'TransONet_Pruned_{PRUNING_RATIO}',
    'dataset': DATASET_NAME,
    'model_parameters': count_parameters(final_model),
    
    # CORE OCCUPANCY METRICS
    'occupancy_accuracy': occupancy_metrics['occupancy_accuracy'],
    'occupancy_precision': occupancy_metrics['occupancy_precision'],
    'occupancy_recall': occupancy_metrics['occupancy_recall'],
    'occupancy_f1': occupancy_metrics['occupancy_f1'],
    'volumetric_iou': occupancy_metrics['volumetric_iou'],

    # MESH QUALITY METRICS
    'mesh_hausdorff_distance': mesh_metrics.get('mesh_hausdorff_distance', None),
    'mesh_chamfer_distance': mesh_metrics.get('mesh_chamfer_distance', None),
    'mesh_volume_error': mesh_metrics.get('mesh_volume_error', None),
    
    # EFFICIENCY METRICS
    'avg_inference_time': occupancy_metrics['avg_inference_time'],
    'throughput': occupancy_metrics['throughput'],
    
    # MODEL-SPECIFIC METRICS
    'prediction_ratio': prediction_ratio,
    'plane_consistency': plane_consistency,
    
    # TRAINING CONFIGURATION
    'batch_size': BATCH_SIZE,
    'epochs_trained': NUM_EPOCHS, # Original epochs
    'finetune_epochs': FINETUNE_EPOCHS, # Fine-tuning epochs
    'learning_rate': LEARNING_RATE,
    'finetune_learning_rate': FINETUNE_LR
}


# --- Display Results in Comparison-Ready Format ---
print(f"\n RESULTS SUMMARY - {pruned_results['approach_name'].upper()}")
print("-" * 60)

print(f"\n CORE METRICS:")
print(f"  Occupancy Accuracy:    {pruned_results['occupancy_accuracy']:.4f}")
print(f"  Occupancy Precision:   {pruned_results['occupancy_precision']:.4f}")
print(f"  Occupancy Recall:      {pruned_results['occupancy_recall']:.4f}")
print(f"  Occupancy F1-Score:    {pruned_results['occupancy_f1']:.4f}")
print(f"  Volumetric IoU:        {pruned_results['volumetric_iou']:.4f}")

print(f"\n MESH QUALITY METRICS:")
if pruned_results['mesh_hausdorff_distance'] is not None:
    print(f"  Hausdorff Distance:    {pruned_results['mesh_hausdorff_distance']:.4f}")
    print(f"  Chamfer Distance:      {pruned_results['mesh_chamfer_distance']:.4f}")
    print(f"  Volume Error:          {pruned_results['mesh_volume_error']:.4f}")
else:
    print("  No ground truth meshes available for evaluation.")

print(f"\n EFFICIENCY METRICS:")
print(f"  Inference Time:        {pruned_results['avg_inference_time']:.4f}s")
print(f"  Throughput:            {pruned_results['throughput']:.2f} samples/s")
print(f"  Model Parameters:      {pruned_results['model_parameters']:,}")

print(f"\n ARCHITECTURE-SPECIFIC:")
print(f"  Prediction Ratio:      {pruned_results['prediction_ratio']:.4f}")
# The plane_consistency is a dictionary, let's print it nicely
if isinstance(pruned_results['plane_consistency'], list):
    avg_consistency = np.mean([p['consistency'] for p in pruned_results['plane_consistency']])
    print(f"  Avg Plane Consistency: {avg_consistency:.4f}")
else:
     print(f"  Plane Consistency:     {pruned_results['plane_consistency']:.4f}")


# --- Save Results for Comparison ---
results_file = f"{RESULTS_PATH}/transonet_finetuned_results.json"
with open(results_file, 'w') as f:
    # A little helper to handle numpy types for JSON serialization
    def convert(o):
        if isinstance(o, np.generic): return o.item()
        raise TypeError
    json.dump(pruned_results, f, indent=2, default=convert)

print(f"\n Fine-tuned results saved to: {results_file}")
print(f" Ready for final approach comparison!")
print("-" * 80)